# TransUNet K-Fold Cross-Validation Training

This notebook implements k-fold cross-validation training and metrics measurement for the **TransUNet model** with **MIFOCAT loss**.

## Overview

- **Model**: TransUNet (Transformer-based U-Net)
- **Loss**: MIFOCAT (MSE + Focal + Categorical Cross-Entropy)
- **Dataset**: ACDC 2017 Cardiac MRI
- **Validation**: 5-fold cross-validation

## Workflow

1. Setup and imports
2. Configure parameters
3. Data splitting (if needed)
4. K-fold training
5. Results aggregation
6. Visualization

## 1. Setup and Imports

In [ ]:
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")

In [ ]:
try:
    import tensorflow as tf
    from tensorflow import keras
    print(f"TensorFlow version: {tf.__version__}")
    
    # Check for GPU availability
    if tf.config.list_physical_devices('GPU'):
        print(f"✓ GPU detected: {tf.config.list_physical_devices('GPU')}")
    else:
        print("⚠ No GPU detected. Using CPU.")
        print(f"  Available devices: {tf.config.list_physical_devices()}")
    
except ImportError as e:
    print(f"TensorFlow import failed: {e}")
    print("Install with: pip install tensorflow keras")

try:
    from split_data import CardiacDataSplitter
    from custom_datagen import FoldAwareDataLoader
    from train_kfold_wrapper import KFoldTrainer
    from transunet_model import build_transunet_mifocat, get_custom_objects
    print("✓ Custom modules imported successfully")
except ImportError as e:
    print(f"Custom module import failed: {e}")
    print("Ensure all required modules are in the project root.")

## 2. Configuration

In [ ]:
# ============ CONFIGURATION ============

# Data paths - ACDC2017 Dataset Structure
DATA_ROOT = Path.cwd() / "acdc2017" / "Data 2D" / "ED" / "Data Per Pasien Training 2D"
OUTPUT_DIR = Path.cwd() / "transunet_kfold_results"
OUTPUT_DIR.mkdir(exist_ok=True)

# K-Fold parameters
N_SPLITS = 5                    # Number of folds
VAL_RATIO = 0.1                 # Validation ratio per fold
RANDOM_SEED = 42

# Training parameters
MODEL_TYPE = "transunet"        # TransUNet model
EPOCHS_PER_FOLD = 50
BATCH_SIZE = 32
EARLY_STOP_PATIENCE = 10
IMAGE_SUBDIR = "images"         # Subdirectory for images within patient folders
MASK_SUBDIR = "groundtruth"     # Subdirectory for masks within patient folders (ACDC2017 uses 'groundtruth')

# Model checkpoint output
CHECKPOINT_DIR = OUTPUT_DIR / "checkpoints"
CHECKPOINT_DIR.mkdir(exist_ok=True)

# Print configuration
print("=" * 60)
print("TRANSUNET K-FOLD TRAINING CONFIGURATION")
print("=" * 60)
print(f"Data root: {DATA_ROOT}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Number of folds: {N_SPLITS}")
print(f"Validation ratio: {VAL_RATIO}")
print(f"Model type: {MODEL_TYPE}")
print(f"Epochs per fold: {EPOCHS_PER_FOLD}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Early stopping patience: {EARLY_STOP_PATIENCE}")
print(f"Mask subdirectory: {MASK_SUBDIR}")
print("=" * 60)

## 3. Data Splitting (Optional)

If you haven't generated the k-fold metadata yet, run this cell. Otherwise, skip to the next section.

In [ ]:
# Uncomment to generate k-fold split metadata
# This only needs to be run once

# splitter = CardiacDataSplitter(
#     input_folder=str(DATA_ROOT),
#     output_folder=str(OUTPUT_DIR)
# )

# print(f"\nExecuting k-fold split with n_splits={N_SPLITS}, val_ratio={VAL_RATIO}...")
# print(f"Data directory: {DATA_ROOT}")
# print(f"Data directory exists: {DATA_ROOT.exists()}")

# splitter.kfold_split(
#     n_splits=N_SPLITS,
#     val_ratio=VAL_RATIO
# )
# print("✓ K-fold split complete")

## 4. Initialize K-Fold Trainer

In [ ]:
# Path to k-fold metadata (generated by split_data.py or above cell)
FOLD_METADATA_PATH = OUTPUT_DIR / "kfold_metadata.json"

if not FOLD_METADATA_PATH.exists():
    print(f"✗ ERROR: Fold metadata not found: {FOLD_METADATA_PATH}")
    print("Run the data splitting step first or provide existing metadata.")
else:
    print(f"✓ Fold metadata found: {FOLD_METADATA_PATH}")
    
    # Initialize trainer
    trainer = KFoldTrainer(
        fold_metadata_path=str(FOLD_METADATA_PATH),
        base_data_dir=str(DATA_ROOT),
        output_dir=str(OUTPUT_DIR),
        seed=RANDOM_SEED
    )
    
    print("✓ KFoldTrainer initialized successfully")

## 5. Run K-Fold Training

This will train the TransUNet model on all folds and save checkpoints + metrics.

In [ ]:
# Run k-fold cross-validation
print("\n🚀 Starting k-fold cross-validation training for TransUNet...\n")

results = trainer.run_all_folds(
    model_type=MODEL_TYPE,
    epochs=EPOCHS_PER_FOLD,
    batch_size=BATCH_SIZE,
    train_only=False,  # Set to True to skip test evaluation
    start_fold=0       # Set to resume from a specific fold
)

print("\n✓ K-fold cross-validation completed!")

## 6. Results Analysis

In [ ]:
# Load and display aggregated results
aggregated_results_path = OUTPUT_DIR / "aggregated_results.json"
fold_results_path = OUTPUT_DIR / "fold_results.json"

if aggregated_results_path.exists():
    with open(aggregated_results_path, 'r') as f:
        aggregated = json.load(f)
    
    print("\n" + "="*60)
    print("AGGREGATED RESULTS (ACROSS ALL FOLDS)")
    print("="*60)
    for key, value in aggregated.items():
        if isinstance(value, float):
            print(f"{key:.<40} {value:.6f}")
        else:
            print(f"{key:.<40} {value}")
    print("="*60)

if fold_results_path.exists():
    with open(fold_results_path, 'r') as f:
        fold_results = json.load(f)
    
    # Create DataFrame for easier visualization
    df = pd.DataFrame(fold_results)
    print("\n" + "="*60)
    print("PER-FOLD RESULTS")
    print("="*60)
    print(df.to_string(index=False))
    print("="*60)

## 7. Visualization

In [ ]:
# Visualize validation loss across folds
if fold_results_path.exists():
    with open(fold_results_path, 'r') as f:
        fold_results = json.load(f)
    
    val_losses = [r.get('final_val_loss') for r in fold_results if r.get('final_val_loss')]
    fold_ids = [r.get('fold_id') for r in fold_results if r.get('final_val_loss')]
    
    if val_losses:
        plt.figure(figsize=(10, 6))
        plt.bar(fold_ids, val_losses, color='steelblue', alpha=0.7)
        plt.axhline(y=np.mean(val_losses), color='red', linestyle='--', label=f'Mean: {np.mean(val_losses):.4f}')
        plt.xlabel('Fold ID', fontsize=12)
        plt.ylabel('Validation Loss', fontsize=12)
        plt.title('TransUNet Validation Loss Across Folds', fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / 'transunet_validation_loss.png', dpi=300)
        plt.show()
        
        print(f"\n✓ Visualization saved to: {OUTPUT_DIR / 'transunet_validation_loss.png'}")

## 8. Load and Test a Specific Fold Model

In [ ]:
# Example: Load best model from fold 0
fold_id = 0
model_path = OUTPUT_DIR / f"fold_{fold_id}" / f"fold_{fold_id}_best_model.h5"

if model_path.exists():
    print(f"Loading model from: {model_path}")
    
    # Load with custom objects
    custom_objs = get_custom_objects()
    model = keras.models.load_model(str(model_path), custom_objects=custom_objs, compile=False)
    
    print("✓ Model loaded successfully")
    print(f"\nModel summary:")
    model.summary()
else:
    print(f"✗ Model not found: {model_path}")

## Summary

This notebook implements k-fold cross-validation training for TransUNet with MIFOCAT loss, mirroring the workflow from `train_kfold_notebook.ipynb` but specifically for the TransUNet architecture.

**Key features:**
- TransUNet model with transformer layers
- MIFOCAT unified loss function
- 5-fold cross-validation
- Per-fold checkpointing
- Aggregated metrics across folds
- Results visualization

**Next steps:**
1. Run detailed evaluation metrics (Dice, IoU, Hausdorff, MCC) on test sets
2. Compare TransUNet results with U-Net baseline
3. Generate prediction visualizations
4. Statistical significance testing